In [9]:
import numpy as np
import pandas as pd
from datetime import datetime 
import csv 
import matplotlib.pyplot as plt
from datetime import date
from datetime import time
import altair as alt

In [35]:
#pd.set_option("max_rows", None)
#pd.reset_option('all')


: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



/Users/selina/opt/anaconda3/lib/python3.7/site-packages/pandas/_config/config.py:622: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


In [10]:
def convertTime(string):
    """Converts a string to a datetime object.
    """
    time = datetime.strptime(string,'%m/%d/%Y %I:%M:%S %p') #Accepts time string, converts it to datetime object.
    return time


def convertMonth(string):
    """Converts a string to a datetime object.
    """
    time = datetime.strptime(string,'%m/%d/%Y') #Accepts time string, converts it to datetime object.
    return time

def convertWeekday(string):
    """Converts a string to a datetime object.
    """
    time = datetime.datetime.strptime(string,'%m/%d/%Y %I:%M:%S %p').strftime('%A') #Accepts time string, converts it to datetime object.
    return time

def convertFromSeconds(s): # total seconds
    """ convertFromSeconds(s): Converts an integer # of seconds into a list of [days, hours, minutes, seconds]
        input s: an int
    """
    s = s*60
    days = s // (24*60*60)  # total days
    s = s % (24*60*60) # remainder s
    hours = s // (60*60) # total hours
    s = s % (60*60) # remainder s
    minutes = s // 60 # total minutes
    s = s % 60 # remainder s
    statement = (days, ' days') + (hours, ' hrs') +(minutes, ' mins') + (s, 'sec')
    return statement
    
def elapsedTime(csv):
    """
    Accepts CSV file and creates new dataframe of all timestamps for both service request submission and fulfillment times.
    Input CSV file in the format: r'filepath'
    Data source: https://data.lacity.org/A-Well-Run-City/MyLA311-Service-Request-Data-2019/pvft-t768
    """
    df = pd.read_csv(csv) #
   
    hdf = df[['CreatedDate','ServiceDate']].dropna() #filter the dataset to the columns CreatedDate and ServiceDate
    
    df1 = hdf['CreatedDate'].apply(convertTime, 0) # apply convertTime to all values in CreatedDate
    df2 = hdf['ServiceDate'].apply(convertTime, 0) # apply convertTime to all values in ServiceDate
    
    df['ElapsedTime'] = df2 - df1 #calculate the difference in the values in CreatedDate and ServiceDate
    df['ElapsedTime']=df['ElapsedTime']/np.timedelta64(1,'m') 
    df['ElapsedTime'] = df['ElapsedTime'].apply(convertFromSeconds, 0) #apply convertFromSeconds to all values in ElapsedTime to 
    #show elapsed time in days, hours, mins, and sec
    
    hdf = df[['CreatedDate','ServiceDate', 'ElapsedTime']].dropna() #create a dataset with the columns CreatedDate, ServiceDate, 
    #and ElapsedTime

    return hdf.head()


In [11]:
df = pd.read_csv("file:///Users/selina/Downloads/MyLA311_Service_Request_Data_2017.csv", low_memory=False)

In [4]:
#df.describe

<bound method NDFrame.describe of             SRNumber             CreatedDate             UpdatedDate  \
0        1-411056231  01/01/2017 12:01:00 AM  01/01/2017 09:26:00 AM   
1        1-411056971  01/01/2017 12:45:00 AM  01/12/2017 05:10:00 PM   
2        1-411057021  01/01/2017 12:46:00 AM  01/01/2017 09:27:00 AM   
3        1-411057091  01/01/2017 01:01:00 AM  01/04/2017 05:33:00 PM   
4        1-411062686  01/01/2017 01:09:00 AM  01/04/2017 05:34:00 PM   
...              ...                     ...                     ...   
1131553  1-870703871  12/31/2017 11:51:00 PM  01/03/2018 10:11:00 AM   
1131554  1-870693641  12/31/2017 11:53:00 PM  01/03/2018 10:12:00 AM   
1131555  1-870694601  12/31/2017 11:56:00 PM  01/03/2018 10:11:00 AM   
1131556  1-870704831  12/31/2017 11:59:00 PM  04/09/2018 09:25:00 AM   
1131557  1-870704814  01/01/2018 12:00:00 AM  01/06/2018 10:18:00 AM   

        ActionTaken Owner          RequestType     Status RequestSource  \
0        SR Created   ITA 

In [5]:
df_1 = df[df['RequestType'] == 'Bulky Items']
df_1.describe

<bound method NDFrame.describe of             SRNumber             CreatedDate             UpdatedDate  \
6        1-411056351  01/01/2017 01:21:00 AM  01/07/2017 09:46:00 AM   
7        1-411057219  01/01/2017 01:26:00 AM  01/04/2017 09:31:00 AM   
8        1-411057302  01/01/2017 01:31:00 AM  01/04/2017 09:51:00 AM   
9        1-411057951  01/01/2017 01:47:00 AM  01/07/2017 08:53:00 AM   
11       1-411062901  01/01/2017 02:00:00 AM  01/04/2017 09:32:00 AM   
...              ...                     ...                     ...   
1131551  1-870699049  12/31/2017 11:50:00 PM  01/02/2018 08:28:00 PM   
1131553  1-870703871  12/31/2017 11:51:00 PM  01/03/2018 10:11:00 AM   
1131554  1-870693641  12/31/2017 11:53:00 PM  01/03/2018 10:12:00 AM   
1131555  1-870694601  12/31/2017 11:56:00 PM  01/03/2018 10:11:00 AM   
1131557  1-870704814  01/01/2018 12:00:00 AM  01/06/2018 10:18:00 AM   

        ActionTaken Owner  RequestType  Status RequestSource MobileOS  \
6        SR Created   BOS  B

In [12]:
hdf = df.filter(['CreatedDate', 'ClosedDate', 'ZipCode', 'RequestType']).dropna()

df1 = hdf['CreatedDate'].apply(convertTime, 0) 
df2 = hdf['ClosedDate'].apply(convertTime, 0)
    
hdf['ElapsedTime'] = df2 - df1
hdf['ElapsedTime']=hdf['ElapsedTime']/np.timedelta64(1,'m') 
    
hdf = hdf[['CreatedDate','ClosedDate', 'ElapsedTime', 'ZipCode', 'RequestType']].dropna() 
hdf.head()

,CreatedDate,ClosedDate,ElapsedTime,ZipCode,RequestType
0,01/01/2017 12:01:00 AM,01/01/2017 09:26:00 AM,565.0,90049,Other
1,01/01/2017 12:45:00 AM,01/12/2017 05:10:00 PM,16825.0,90026,Graffiti Removal
2,01/01/2017 12:46:00 AM,01/01/2017 09:27:00 AM,521.0,90049,Other
3,01/01/2017 01:01:00 AM,01/04/2017 05:33:00 PM,5312.0,91311,Graffiti Removal
4,01/01/2017 01:09:00 AM,01/04/2017 05:34:00 PM,5305.0,91311,Graffiti Removal


In [13]:
import datetime
df_test = hdf['ClosedDate'].apply(convertWeekday, 0)

In [14]:
hdf['Weekday'] = df_test
hdf.head()

,CreatedDate,ClosedDate,ElapsedTime,ZipCode,RequestType,Weekday
0,01/01/2017 12:01:00 AM,01/01/2017 09:26:00 AM,565.0,90049,Other,Sunday
1,01/01/2017 12:45:00 AM,01/12/2017 05:10:00 PM,16825.0,90026,Graffiti Removal,Thursday
2,01/01/2017 12:46:00 AM,01/01/2017 09:27:00 AM,521.0,90049,Other,Sunday
3,01/01/2017 01:01:00 AM,01/04/2017 05:33:00 PM,5312.0,91311,Graffiti Removal,Wednesday
4,01/01/2017 01:09:00 AM,01/04/2017 05:34:00 PM,5305.0,91311,Graffiti Removal,Wednesday


In [32]:
grouped = hdf.groupby(['RequestType']).mean()
#grouped = grouped.sort_index(ascending=False)
grouped.sort_values(['ElapsedTime'], ascending = False)
#grouped

,ElapsedTime
RequestType,
Feedback,125272.657459
Multiple Streetlight Issue,73971.597535
Homeless Encampment,71060.710636
Single Streetlight Issue,21227.477425
Illegal Dumping Pickup,11599.282923
Bulky Items,5295.223050
Metal/Household Appliances,4896.357150
Electronic Waste,4826.700583
Other,4716.050111


In [42]:
df_2 = hdf[hdf['RequestType'] == 'Feedback']
df_2

,CreatedDate,ClosedDate,ElapsedTime,ZipCode,RequestType,Weekday
39,01/01/2017 07:54:00 AM,01/09/2017 04:49:00 PM,12055.0,90004,Feedback,Monday
42,01/01/2017 07:58:00 AM,01/03/2017 09:50:00 AM,2992.0,90004,Feedback,Tuesday
5932,01/03/2017 03:57:00 PM,01/10/2017 10:58:00 AM,9781.0,90020,Feedback,Tuesday
13951,01/05/2017 09:36:00 PM,01/05/2017 09:36:00 PM,0.0,91367,Feedback,Thursday
16047,01/06/2017 04:50:00 PM,01/06/2017 04:50:00 PM,0.0,90059,Feedback,Friday
...,...,...,...,...,...,...
1106229,12/20/2017 11:51:00 AM,12/21/2017 12:12:00 AM,741.0,90004,Feedback,Thursday
1108271,12/20/2017 07:53:00 PM,12/21/2017 12:20:00 AM,267.0,90020,Feedback,Thursday
1109066,12/21/2017 09:16:00 AM,06/03/2018 10:07:00 AM,236211.0,90020,Feedback,Sunday
1111052,12/21/2017 04:00:00 PM,12/22/2017 06:41:00 AM,881.0,90020,Feedback,Friday


In [21]:
hdf.groupby(['Weekday']).count()

,CreatedDate,ClosedDate,ElapsedTime,ZipCode,RequestType
Weekday,,,,,
Friday,211008,211008,211008,211008,211008
Monday,183611,183611,183611,183611,183611
Saturday,40050,40050,40050,40050,40050
Sunday,14990,14990,14990,14990,14990
Thursday,235858,235858,235858,235858,235858
Tuesday,222341,222341,222341,222341,222341
Wednesday,221572,221572,221572,221572,221572


In [43]:
#d = hdf.groupby('ZipCode')['RequestType']['Weekday'].value_counts(ascending = False, normalize = True)
#d.tail(50)
d = hdf.groupby(["ZipCode", "RequestType"])["Weekday"].value_counts(ascending = False)
d

ZipCode                        RequestType          Weekday  
0                              Bulky Items          Monday       4
                                                    Tuesday      4
                                                    Wednesday    2
                                                    Thursday     1
                               Dead Animal Removal  Tuesday      2
                                                                ..
91803                          Graffiti Removal     Friday       1
                                                    Sunday       1
                                                    Wednesday    1
JR BLVD AT AVALON BLVD, 90011  Graffiti Removal     Tuesday      1
JR BLVD AT WESTERN AVE, 90062  Graffiti Removal     Friday       1
Name: Weekday, Length: 7996, dtype: int64

In [65]:
hdf.CreatedDate.str.startswith('12').sum()

84817

In [66]:
jan_17 = hdf[:86566]
feb_17 = hdf[86567:164396]
mar_17 = hdf[164396:263263]
apr_17 = hdf[263264:350314]
may_17 = hdf[350315:446755]
jun_17 = hdf[446756:545216]
jul_17 = hdf[545217:648726]
aug_17 = hdf[648727:762510]
sep_17 = hdf[762511:863193]
oct_17 = hdf[863194:960093]
nov_17 = hdf[960094:1046551]
dec_17 = hdf[1046552:1131368]


In [67]:
df1 = pd.read_csv("file:///Users/selina/Downloads/MyLA311_Service_Request_Data_2018.csv", low_memory=False)

In [68]:
hdf1 = df1.filter(['CreatedDate', 'ClosedDate']).dropna()
df3 = hdf1['CreatedDate'].apply(convertTime, 0)
df4 = hdf1['ClosedDate'].apply(convertTime, 0)

hdf1['ElapsedTime'] = df4 - df3 
hdf1['ElapsedTime']=hdf1['ElapsedTime']/np.timedelta64(1,'m') 
    
hdf1 = hdf1[['CreatedDate','ClosedDate', 'ElapsedTime']].dropna()

hdf1.head()

,CreatedDate,ClosedDate,ElapsedTime
0,01/01/2018 12:00:00 AM,01/06/2018 10:18:00 AM,7818.0
1,01/01/2018 12:05:00 AM,01/03/2018 12:14:00 PM,3609.0
2,01/01/2018 12:08:00 AM,01/02/2018 10:10:00 PM,2762.0
3,01/01/2018 12:09:00 AM,01/03/2018 12:52:00 PM,3643.0
4,01/01/2018 12:10:00 AM,01/06/2018 09:52:00 AM,7782.0


In [69]:
hdf1.CreatedDate.str.startswith('12').sum()

90132

In [70]:
jan_18 = hdf1[:101387]
feb_18 = hdf1[101388:186630]
mar_18 = hdf1[186631:283969]
apr_18 = hdf1[283970:383502]
may_18 = hdf1[383503:485015]
jun_18 = hdf1[485016:587060]
jul_18 = hdf1[587061:698806]
aug_18 = hdf1[698807:812527]
sep_18 = hdf1[812528:915221]
oct_18 = hdf1[915221:1026368]
nov_18 = hdf1[1026369:1118895]
dec_18 = hdf1[1118896:]

In [71]:
dec_18.tail()

,CreatedDate,ClosedDate,ElapsedTime
1210070,12/31/2018 11:40:00 PM,01/02/2019 09:55:00 AM,2055.0
1210071,12/31/2018 11:43:00 PM,01/02/2019 11:10:00 AM,2127.0
1210072,12/31/2018 11:50:00 PM,01/15/2019 11:11:00 AM,20841.0
1210073,12/31/2018 11:53:00 PM,01/04/2019 02:37:00 PM,5204.0
1210074,12/31/2018 11:55:00 PM,01/02/2019 10:20:00 PM,2785.0


In [72]:
df_19 = pd.read_csv("file:///Users/selina/Downloads/MyLA311_Service_Request_Data_2019.csv", low_memory=False)

In [73]:
hdf_19 = df_19.filter(['CreatedDate', 'ClosedDate']).dropna()
df_191 = hdf_19['CreatedDate'].apply(convertTime, 0)
df_192 = hdf_19['ClosedDate'].apply(convertTime, 0)

hdf_19['ElapsedTime'] = df_192 - df_191 
hdf_19['ElapsedTime']=hdf_19['ElapsedTime']/np.timedelta64(1,'m') 
    
hdf_19 = hdf_19[['CreatedDate','ClosedDate', 'ElapsedTime']].dropna()

hdf_19.head()

,CreatedDate,ClosedDate,ElapsedTime
0,01/01/2019 12:02:00 AM,01/04/2019 11:03:00 AM,4981.0
1,01/01/2019 12:05:00 AM,01/01/2019 12:05:00 AM,0.0
2,01/01/2019 12:10:00 AM,01/03/2019 12:27:00 AM,2897.0
3,01/01/2019 12:19:00 AM,01/07/2019 09:39:00 AM,9200.0
4,01/01/2019 12:28:00 AM,01/03/2019 12:28:00 AM,2880.0


In [74]:
hdf_19['Month'] = hdf_19['CreatedDate'].str.split(' ', expand = True)[0]

In [75]:
hdf_19['Month'] = hdf_19['Month'].apply(convertMonth,0)

In [140]:
hdf_19

,CreatedDate,ClosedDate,ElapsedTime,Month
0,01/01/2019 12:02:00 AM,01/04/2019 11:03:00 AM,4981.0,2019-01-01
1,01/01/2019 12:05:00 AM,01/01/2019 12:05:00 AM,0.0,2019-01-01
2,01/01/2019 12:10:00 AM,01/03/2019 12:27:00 AM,2897.0,2019-01-01
3,01/01/2019 12:19:00 AM,01/07/2019 09:39:00 AM,9200.0,2019-01-01
4,01/01/2019 12:28:00 AM,01/03/2019 12:28:00 AM,2880.0,2019-01-01
...,...,...,...,...
1308088,12/31/2019 11:26:00 PM,01/03/2020 04:09:00 PM,3883.0,2019-12-31
1308089,12/31/2019 11:26:00 PM,01/14/2020 07:51:00 AM,19225.0,2019-12-31
1308090,12/31/2019 11:30:00 PM,01/03/2020 11:36:00 AM,3606.0,2019-12-31
1308091,12/31/2019 11:54:00 PM,01/06/2020 10:46:00 AM,7852.0,2019-12-31


In [88]:
g = hdf_19.groupby(pd.Grouper(key='Month', freq='M'))
dfs = [group for _,group in g]
dfs[11]

jan_19 = dfs[0]
feb_19 = dfs[1]
mar_19 = dfs[2]
apr_19 = dfs[3]
may_19 = dfs[4]
jun_19 = dfs[5]
jul_19 = dfs[6]
aug_19 = dfs[7]
sep_19 = dfs[8]
oct_19 = dfs[9]
nov_19 = dfs[10]
dec_19 = dfs[11]

In [115]:
lst = ['jan_17', 'jan_18', 'feb_17', 'feb_18', 'mar_17', 'mar_18', 'apr_17', 'apr_18', 'may_17', 'may_18', 'jun_17', 'jun_18', 'jul_17', 'jul_18','aug_17', 'aug_18','sep_17','sep_18', 'oct_17','oct_18', 'nov_17','nov_18', 'dec_17', 'dec_18']
lsts = [jan_17, jan_18, feb_17, feb_18, mar_17, mar_18, apr_17, apr_18, may_17, may_18, jun_17, jun_18, jul_17, jul_18, aug_17, aug_18, sep_17, sep_18, oct_17, oct_18, nov_17, nov_18, dec_17, dec_18]

lst2 = ['jan_18', 'jan_19', 'feb_18', 'feb_19', 'mar_18', 'mar_19', 'apr_18', 'apr_19', 'may_18', 'may_19', 'jun_18', 'jun_19', 'jul_18', 'jul_19','aug_18', 'aug_19','sep_18','sep_19', 'oct_18','oct_19', 'nov_18','nov_19', 'dec_18', 'dec_19']
lsts_2 = [jan_18, jan_19, feb_18, feb_19, mar_18, mar_19, apr_18, apr_19, may_18, may_19, jun_18, jun_19, jul_18, jul_19, aug_18, aug_19, sep_18, sep_19, oct_18, oct_19, nov_18, nov_19, dec_18, dec_19]

lst1 = ['jan_17', 'feb_17', 'mar_17', 'apr_17','may_17', 'jun_17', 'jul_17', 'aug_17', 'sep_17','oct_17', 'nov_17','dec_17']
lsts1 = [jan_17, feb_17, mar_17, apr_17, may_17, jun_17, jul_17, aug_17, sep_17, oct_17, nov_17, dec_17]

lst_18 = ['jan_18','feb_18', 'mar_18', 'apr_18', 'may_18', 'jun_18', 'jul_18', 'aug_18', 'sep_18','oct_18', 'nov_18', 'dec_18']
lsts_18 = [jan_18, feb_18, mar_18, apr_18, may_18, jun_18, jul_18, aug_18, sep_18, oct_18, nov_18, dec_18]

lst_19 = ['jan_19', 'feb_19', 'mar_19', 'apr_19','may_19', 'jun_19', 'jul_19', 'aug_19', 'sep_19','oct_19', 'nov_19','dec_19']
lsts_19 = [jan_19, feb_19, mar_19, apr_19, may_19, jun_19, jul_19, aug_19, sep_19, oct_19, nov_19, dec_19]

#chart for average elapsed time comparing month-by-month 2017 and 2018

avgs = []

for month in lsts:
    val = month['ElapsedTime'].mean()
    avgs.append(val)
    
res = dict(zip(lst, avgs))
res

df_comp = pd.DataFrame(list(res.items()),columns = ['months','avg_elapsed_time'])
df_comp

alt.Chart(df_comp).mark_bar().encode(
    x=alt.X('months', sort=lst),
    y='avg_elapsed_time'
)

alt.Chart(...)

In [120]:
#chart for average elapsed time comparing month-by-month 2018 and 2019

avgs2 = []

for month in lsts_2:
    val = month['ElapsedTime'].mean()
    avgs2.append(val)
    
res_1819 = dict(zip(lst2, avgs))
res_1819

df_comp_1819 = pd.DataFrame(list(res_1819.items()),columns = ['months','avg_elapsed_time'])
df_comp_1819

alt.Chart(df_comp_1819).mark_bar().encode(
    x=alt.X('months', sort=lst2),
    y='avg_elapsed_time'
)

alt.Chart(...)

In [85]:
#chart for average elapsed time per month in 2017

avgs_17 = []

for month in lsts1:
    val = month['ElapsedTime'].mean()
    avgs_17.append(val)
    
res_17 = dict(zip(lst1, avgs_17))
res_17

df_comp_17 = pd.DataFrame(list(res_17.items()),columns = ['months','avg_elapsed_time'])
df_comp_17

alt.Chart(df_comp_17).mark_bar().encode(
    x=alt.X('months', sort=lst),
    y='avg_elapsed_time'
)

alt.Chart(...)

In [86]:
#chart for average elapsed time per month in 2018

avgs_18 = []

for month in lsts_18:
    val = month['ElapsedTime'].mean()
    avgs_18.append(val)
    
res_18 = dict(zip(lst_18, avgs_18))
res_18

df_comp_18 = pd.DataFrame(list(res_18.items()),columns = ['months','avg_elapsed_time'])
df_comp_18

alt.Chart(df_comp_18).mark_bar().encode(
    x=alt.X('months', sort=lst),
    y='avg_elapsed_time'
)

alt.Chart(...)

In [118]:
#chart for average elapsed time per month in 2019

avgs_19 = []

for month in lsts_19:
    val = month['ElapsedTime'].mean()
    avgs_19.append(val)
    
res_19 = dict(zip(lst_19, avgs_19))
res_19

df_comp_19 = pd.DataFrame(list(res_19.items()),columns = ['months','avg_elapsed_time'])
df_comp_19

alt.Chart(df_comp_19).mark_bar().encode(
    x=alt.X('months', sort=lst),
    y='avg_elapsed_time'
)

alt.Chart(...)

In [80]:
#chart comparing month-by-month number of requests in 2017 and 2018

freqs = []

for month in lsts:
    val = len(month['ElapsedTime'])
    freqs.append(val)
    
res1 = dict(zip(lst, freqs))
res1

df_comp1 = pd.DataFrame(list(res1.items()),columns = ['months','number_of_requests'])
df_comp1


alt.Chart(df_comp1).mark_bar().encode(
    x=alt.X('months', sort=lst),
    y='number_of_requests')

alt.Chart(...)

In [121]:
#chart comparing month-by-month number of requests in 2018 and 2019

freqs_ = []

for month in lsts_2:
    val = len(month['ElapsedTime'])
    freqs_.append(val)
    
res_ = dict(zip(lst2, freqs))
res_

df_comp_ = pd.DataFrame(list(res_.items()),columns = ['months','number_of_requests'])
df_comp_


alt.Chart(df_comp_).mark_bar().encode(
    x=alt.X('months', sort=lst2),
    y='number_of_requests')

alt.Chart(...)

In [122]:
#chart for number of requests by month in 2017
freqss = []

for month in lsts1:
    val = len(month['ElapsedTime'])
    freqss.append(val)
    
res2 = dict(zip(lst1, freqss))
res2

df_comp2 = pd.DataFrame(list(res2.items()),columns = ['months','number_of_requests'])
df_comp2


alt.Chart(df_comp2).mark_bar().encode(
    x=alt.X('months', sort=lst),
    y='number_of_requests')

alt.Chart(...)

In [82]:
#chart for number of requests by month in 2018
freqs_18 = []

for month in lsts_18:
    val = len(month['ElapsedTime'])
    freqs_18.append(val)
    
res3 = dict(zip(lst_18, freqs_18))
res3

df_comp3 = pd.DataFrame(list(res3.items()),columns = ['months','number_of_requests'])
df_comp3


alt.Chart(df_comp3).mark_bar().encode(
    x=alt.X('months', sort=lst),
    y='number_of_requests')

alt.Chart(...)

In [123]:
#chart for number of requests by month in 2019
freqs_19 = []

for month in lsts_19:
    val = len(month['ElapsedTime'])
    freqs_19.append(val)
    
res_19 = dict(zip(lst_19, freqs_19))
res_19

df_comp_19 = pd.DataFrame(list(res_19.items()),columns = ['months','number_of_requests'])
df_comp_19


alt.Chart(df_comp_19).mark_bar().encode(
    x=alt.X('months', sort=lst),
    y='number_of_requests')

alt.Chart(...)